In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from keras.models import Sequential
from keras import Input
import pandas as pd
from sklearn.model_selection import train_test_split
import copy
import collections
from random import shuffle
import itertools
from os import listdir
import random
import string
import statistics
import pickle
from pathlib import Path
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error

2025-12-09 23:23:29.090415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-09 23:23:29.114989: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-09 23:23:29.341696: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-09 23:23:29.343122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-09 23:23:29.886973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [2]:
print(f"TensorFlow: {tf.__version__}")
print(f"NumPy: {np.__version__}")

TensorFlow: 2.13.1
NumPy: 1.23.5


In [3]:
columns = [
    'Weight on Bit (klbs)',
    'Rotary RPM (RPM)',
    'Total Pump Output (gal_per_min)',
    'Rate Of Penetration (ft_per_hr)',
    'Standpipe Pressure (psi)',
    'Rotary Torque (kft_lb)', 
    'Hole Depth (feet)', 
    'Bit Depth (feet)'
]

In [4]:
def csv_to_windows(dataset, columns):
    df = pd.read_csv(os.path.join("Datasets", "MaskedAutoencoder", dataset))
    df = df[columns]

    base_mask = (
        (df["Hole Depth (feet)"].rolling(10000).mean().diff() > 0) &
        (df["Hole Depth (feet)"] == df["Bit Depth (feet)"]) &
        (df["Hole Depth (feet)"] > 1000)
    )
    
    window = 100       # Rolling window size
    threshold = 0.3    # Keep if rolling average > threshold
    
    # Compute rolling average of the mask (convert to 0/1 first)
    rolling_avg = base_mask.astype(float).rolling(window).mean()
    
    # Final mask based on rolling average threshold
    final_mask = (rolling_avg > threshold).fillna(0)
    
    final_mask = final_mask.astype(float).rolling(20000).mean() > 0.6
    
    masked_hole_depth = df["Hole Depth (feet)"].where(final_mask, np.nan)
    
    gap_threshold = 100  # maximum number of consecutive NaNs to merge segments
    
    # Identify indices of non-NaN values
    not_nan_idx = masked_hole_depth[masked_hole_depth.notna()].index
    
    # Grouping non-NaN indices based on closeness
    groups = []
    current_group = []
    
    for i, idx in enumerate(not_nan_idx):
        if i == 0:
            current_group.append(idx)
            continue
    
        # Check gap from previous index
        if idx - not_nan_idx[i-1] <= gap_threshold:
            current_group.append(idx)
        else:
            groups.append(current_group)
            current_group = [idx]
    
    # Append last group
    if current_group:
        groups.append(current_group)

    # Fix all NaNs
    drilling_segments = [  ]
    window_size = 100
    for group in groups:
        dfg = df.loc[group].copy()
        
        for col in dfg.columns:
            if np.issubdtype(dfg[col].dtype, np.number):
                series = dfg[col]      
                rolling_mean = series.rolling(window=window_size, min_periods=1, center=True).mean()
                dfg[col] = series.fillna(rolling_mean).bfill(  ).ffill()
    
        drilling_segments.append(dfg)
    
    # Min Max Normalization
    global_min = pd.concat(drilling_segments).min()
    global_max = pd.concat(drilling_segments).max()
    
    # Step 2: Normalize each dataframe
    print(f"Drilling Segments: {len(drilling_segments)}")
    normalized_drilling_segments = []
    for df in drilling_segments:
        normalized_df = (df - global_min) / (global_max - global_min)
        normalized_drilling_segments.append(normalized_df)

    window_size = 60 * 10  # 10 minutes

    windows = []
    count = 1
    for df in normalized_drilling_segments:
        print(f"\t{count}")
        count += 1
        for i in range(len(df) - window_size + 1):
            window = df.iloc[i:i + window_size]
            windows.append(window.to_numpy())

    print(f"Windows: {len(windows):,}".replace(',', ' ')) 
    print(f"Windows per Segment: {len(windows) / len(drilling_segments):,.2f}".replace(',', ' '))
    
    return windows

In [5]:
def mask_data(data, MASKING_PERCENT=0.8):
    masked_data = []
    
    for i in range(len(data)):
        if i % 10000 == 0:
            print(f"Processing: {i}/{len(data)} ({100*i/len(data):.1f}%)")
    
        arr = np.array(data[i], dtype=np.float32)  # Use float32 instead of float64
        masked_arr = arr.copy()
        
        total_elements = arr.size
        n_mask = int(total_elements * MASKING_PERCENT)
        
        flat_indices = np.random.choice(total_elements, size=n_mask, replace=False)
        mask_indices = np.unravel_index(flat_indices, arr.shape)
        
        masked_arr[mask_indices] = 0  # Use 0 instead of NaN for neural networks
        
        masked_data.append(masked_arr)
    
    return np.array(masked_data, dtype=np.float32)  # Return as numpy array

In [6]:
class MaskedDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size=32, mask_percent=0.8, shuffle=True):
        self.data = np.array(data, dtype=np.float32)
        self.batch_size = batch_size
        self.mask_percent = mask_percent
        self.shuffle = shuffle
        self.indices = np.arange(len(data))
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))
    
    def __getitem__(self, idx):
        # Get batch indices
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.data[batch_indices]
        
        # Create masked version (X) from original (Y)
        batch_x = batch_y.copy()
        for i in range(len(batch_x)):
            n_mask = int(batch_x[i].size * self.mask_percent)
            flat_indices = np.random.choice(batch_x[i].size, size=n_mask, replace=False)
            mask_indices = np.unravel_index(flat_indices, batch_x[i].shape)
            batch_x[i][mask_indices] = 0
            
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [7]:
# Autoencoder training: 78B-32 1 sec data 27200701.csv, 27029986-3.csv
# Task Header 1 (DAS Stickslip): 27029986-4.csv
# Task Header 2 (Temp OUT (Degrees)): 27029986-5.csv

In [ ]:
windows1 = csv_to_windows("27029986-3.csv", columns)
windows2 = csv_to_windows("78B-32 1 sec data 27200701.csv", columns)

# Shuffle both lists
random.seed(42)
random.shuffle(windows1)
random.shuffle(windows2)

# Take the same amount from each (the minimum length)
min_length = min(len(windows1), len(windows2))
windows1_sampled = windows1[:min_length]
windows2_sampled = windows2[:min_length]

# Combine them
windows = windows1_sampled + windows2_sampled

# Shuffle the combined list
random.shuffle(windows)

print(f"Sampled {min_length:,} from each list".replace(',', ' '))
print(f"Total windows: {len(windows):,}".replace(',', ' '))

Drilling Segments: 3
	1
	2
	3
Windows: 131 072
Windows per Segment: 43 690.67
Drilling Segments: 9
	1
	2
	3
	4
	5
	6
	7


In [ ]:
SUBSET_PERCENT = 0.3  # Use 30% of windows (adjust as needed)

# Calculate how many windows to keep
n_windows_to_keep = int(len(windows) * SUBSET_PERCENT)

# Randomly sample indices
random.seed(42)
subset_indices = random.sample(range(len(windows)), n_windows_to_keep)

# Select the subset
windows = [windows[i] for i in subset_indices]

print(f"📊 Subset Selection:")
print(f"   Original windows: {len(windows) / SUBSET_PERCENT:,.0f}".replace(',', ' '))
print(f"   Subset percent: {SUBSET_PERCENT * 100:.1f}%")
print(f"   Selected windows: {len(windows):,}".replace(',', ' '))
print(f"   Estimated memory reduction: {(1 - SUBSET_PERCENT) * 100:.1f}%")


In [ ]:
# Cell 8 - Split into train/test and convert to numpy ONCE
train_windows, test_windows = train_test_split(windows, test_size=0.2, random_state=42)

# Convert to numpy arrays (only store Y, not X)
train_windows_y = np.array(train_windows, dtype=np.float32)
test_windows_y = np.array(test_windows, dtype=np.float32)

# Free up memory
del windows, windows1, windows2, windows1_sampled, windows2_sampled, train_windows, test_windows

print(f"Train shape: {train_windows_y.shape}")
print(f"Test shape: {test_windows_y.shape}")
print(f"Memory for train_y: {train_windows_y.nbytes / 1e9:.2f} GB")
print(f"Memory for test_y: {test_windows_y.nbytes / 1e9:.2f} GB")

In [ ]:
# Cell 9 - Create generators (NO masking done here, just setup)
train_gen = MaskedDataGenerator(train_windows_y, batch_size=32, mask_percent=0.8, shuffle=True)
test_gen = MaskedDataGenerator(test_windows_y, batch_size=32, mask_percent=0.8, shuffle=False)

print(f"Train batches per epoch: {len(train_gen)}")
print(f"Test batches: {len(test_gen)}")

# Test that it works
batch_x, batch_y = train_gen[0]
print(f"Batch X shape: {batch_x.shape}")
print(f"Batch Y shape: {batch_y.shape}")
print(f"Masking working: {np.sum(batch_x == 0) > 0}")

In [ ]:
# Fixed MetricsCallback - Memory efficient without K.clear_session()

class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_gen, test_gen, subset_fraction=0.05):
        """
        Memory-efficient metrics callback
        
        Args:
            train_gen: Training data generator
            test_gen: Test data generator
            subset_fraction: Fraction of batches to evaluate (default: 0.05 = 5%)
        """
        super().__init__()
        self.train_gen = train_gen
        self.test_gen = test_gen
        self.subset_fraction = subset_fraction
        self.history = {
            'train_rmse': [],
            'train_mae': [],
            'test_rmse': [],
            'test_mae': []
        }
    
    def on_epoch_end(self, epoch, logs=None):
        import gc
        
        print(f"\n📊 Calculating metrics for epoch {epoch + 1}...")
        print(f"   (Evaluating on {self.subset_fraction*100:.1f}% of batches)")
        
        # Calculate number of batches to evaluate
        max_train_batches = max(1, int(len(self.train_gen) * self.subset_fraction))
        max_test_batches = max(1, int(len(self.test_gen) * self.subset_fraction))
        
        # === TRAINING METRICS ===
        train_mse_list = []
        train_mae_list = []
        
        print(f"   Training: 0/{max_train_batches}", end="", flush=True)
        
        for i in range(max_train_batches):
            # Get batch
            batch_x, batch_y = self.train_gen[i]
            
            # Predict (use predict_on_batch for efficiency)
            batch_pred = self.model.predict_on_batch(batch_x)
            
            # Calculate metrics and store
            train_mse_list.append(np.mean((batch_y - batch_pred) ** 2))
            train_mae_list.append(np.mean(np.abs(batch_y - batch_pred)))
            
            # Immediately delete to free memory
            del batch_x, batch_y, batch_pred
            
            # Progress update
            if (i + 1) % 50 == 0 or (i + 1) == max_train_batches:
                print(f"\r   Training: {i+1}/{max_train_batches}", end="", flush=True)
                gc.collect()
        
        # Calculate final metrics
        train_rmse = np.sqrt(np.mean(train_mse_list))
        train_mae = np.mean(train_mae_list)
        print(f"\r   Training: {max_train_batches}/{max_train_batches} - RMSE: {train_rmse:.6f}, MAE: {train_mae:.6f}")
        
        # Clear lists
        del train_mse_list, train_mae_list
        gc.collect()
        
        # === TEST METRICS ===
        test_mse_list = []
        test_mae_list = []
        
        print(f"   Testing: 0/{max_test_batches}", end="", flush=True)
        
        for i in range(max_test_batches):
            # Get batch
            batch_x, batch_y = self.test_gen[i]
            
            # Predict
            batch_pred = self.model.predict_on_batch(batch_x)
            
            # Calculate metrics and store
            test_mse_list.append(np.mean((batch_y - batch_pred) ** 2))
            test_mae_list.append(np.mean(np.abs(batch_y - batch_pred)))
            
            # Immediately delete
            del batch_x, batch_y, batch_pred
            
            # Progress update
            if (i + 1) % 25 == 0 or (i + 1) == max_test_batches:
                print(f"\r   Testing: {i+1}/{max_test_batches}", end="", flush=True)
                gc.collect()
        
        # Calculate final metrics
        test_rmse = np.sqrt(np.mean(test_mse_list))
        test_mae = np.mean(test_mae_list)
        print(f"\r   Testing: {max_test_batches}/{max_test_batches} - RMSE: {test_rmse:.6f}, MAE: {test_mae:.6f}")
        
        # Clear lists
        del test_mse_list, test_mae_list
        gc.collect()
        
        # Store metrics as Python floats (not numpy types)
        self.history['train_rmse'].append(float(train_rmse))
        self.history['train_mae'].append(float(train_mae))
        self.history['test_rmse'].append(float(test_rmse))
        self.history['test_mae'].append(float(test_mae))
        
        print(f"   ✅ Epoch {epoch + 1} complete")


# Create the callback with 5% evaluation (reduced from 10%)
metrics_callback = MetricsCallback(
    train_gen, 
    test_gen,
    subset_fraction=0.05  # Only evaluate 5% of batches
)

print("✅ Memory-efficient callback created!")
print(f"   Will evaluate ~{int(len(train_gen)*0.05)} training batches and ~{int(len(test_gen)*0.05)} test batches per epoch")

In [ ]:
# Cell 10 - Build model
model = Sequential()
model.add(LSTM(128, activation='tanh', input_shape=(600, 8), return_sequences=True))
model.add(LSTM(64, activation='tanh', return_sequences=False))
model.add(RepeatVector(600))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(LSTM(128, activation='tanh', return_sequences=True))
model.add(TimeDistributed(Dense(8)))

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Cell 11 - Train model with the callback
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen,
    callbacks=[metrics_callback],  # Add the callback here
    verbose=1
)

In [ ]:
# Plot training loss
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation loss', marker='o')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training History - Loss')
plt.grid(True)

# Plot RMSE
plt.subplot(1, 3, 2)
plt.plot(metrics_callback.history['train_rmse'], label='Train RMSE', marker='o')
plt.plot(metrics_callback.history['test_rmse'], label='Test RMSE', marker='o')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Training History - RMSE')
plt.grid(True)

# Plot MAE
plt.subplot(1, 3, 3)
plt.plot(metrics_callback.history['train_mae'], label='Train MAE', marker='o')
plt.plot(metrics_callback.history['test_mae'], label='Test MAE', marker='o')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Training History - MAE')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 12 - Display final metrics table
print("\n" + "=" * 70)
print("METRICS PER EPOCH")
print("=" * 70)
print(f"{'Epoch':<8} {'Train RMSE':<15} {'Train MAE':<15} {'Test RMSE':<15} {'Test MAE':<15}")
print("-" * 70)

for i in range(len(metrics_callback.history['train_rmse'])):
    print(f"{i+1:<8} "
          f"{metrics_callback.history['train_rmse'][i]:<15.6f} "
          f"{metrics_callback.history['train_mae'][i]:<15.6f} "
          f"{metrics_callback.history['test_rmse'][i]:<15.6f} "
          f"{metrics_callback.history['test_mae'][i]:<15.6f}")

print("=" * 70)

# Final epoch summary
final_epoch = len(metrics_callback.history['train_rmse']) - 1
print(f"\n📊 Final Epoch ({final_epoch + 1}) Summary:")
print(f"   Training   - RMSE: {metrics_callback.history['train_rmse'][final_epoch]:.6f}, "
      f"MAE: {metrics_callback.history['train_mae'][final_epoch]:.6f}")
print(f"   Test       - RMSE: {metrics_callback.history['test_rmse'][final_epoch]:.6f}, "
      f"MAE: {metrics_callback.history['test_mae'][final_epoch]:.6f}")

In [ ]:
# Get a few random test samples
np.random.seed(42)
num_examples = 3
random_indices = np.random.choice(len(test_windows_y), num_examples, replace=False)

# Get the samples
sample_y = test_windows_y[random_indices]

# Create masked versions and get predictions
sample_x = sample_y.copy()
for i in range(len(sample_x)):
    n_mask = int(sample_x[i].size * 0.8)
    flat_indices = np.random.choice(sample_x[i].size, size=n_mask, replace=False)
    mask_indices = np.unravel_index(flat_indices, sample_x[i].shape)
    sample_x[i][mask_indices] = 0

# Get predictions
predictions = model.predict(sample_x, verbose=0)

# Feature names for labeling
feature_names = [
    'Weight on Bit',
    'Rotary RPM',
    'Total Pump Output',
    'Rate Of Penetration',
    'Standpipe Pressure',
    'Rotary Torque',
    'Hole Depth',
    'Bit Depth'
]

# Create visualization for each example
for example_idx in range(num_examples):
    fig, axes = plt.subplots(8, 1, figsize=(16, 20))
    fig.suptitle(f'Example {example_idx + 1}: Reconstruction Quality', fontsize=16, fontweight='bold')
    
    original = sample_y[example_idx]
    masked = sample_x[example_idx]
    reconstructed = predictions[example_idx]
    
    # Plot each feature
    for feature_idx in range(8):
        ax = axes[feature_idx]
        
        # Original data (ground truth)
        ax.plot(original[:, feature_idx], label='Original (Ground Truth)', 
                color='green', linewidth=2, alpha=0.8)
        
        # Masked input (what the model sees)
        masked_feature = masked[:, feature_idx].copy()
        masked_feature[masked_feature == 0] = np.nan  # Show gaps where masked
        ax.plot(masked_feature, label='Masked Input', 
                color='red', linewidth=1.5, alpha=0.6, linestyle='--')
        
        # Reconstructed output
        ax.plot(reconstructed[:, feature_idx], label='Reconstructed', 
                color='blue', linewidth=2, alpha=0.7)
        
        # Calculate error for this feature
        feature_mae = mean_absolute_error(original[:, feature_idx], 
                                         reconstructed[:, feature_idx])
        feature_rmse = np.sqrt(mean_squared_error(original[:, feature_idx], 
                                                  reconstructed[:, feature_idx]))
        
        ax.set_title(f'{feature_names[feature_idx]} - MAE: {feature_mae:.4f}, RMSE: {feature_rmse:.4f}', 
                    fontsize=10, fontweight='bold')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Normalized Value')
        ax.legend(loc='upper right', fontsize=8)
        ax.grid(True, alpha=0.3)
        ax.set_ylim(-0.1, 1.1)
    
    plt.tight_layout()
    plt.show()
    
    # Print overall metrics for this example
    example_mae = mean_absolute_error(original.reshape(-1), reconstructed.reshape(-1))
    example_rmse = np.sqrt(mean_squared_error(original.reshape(-1), reconstructed.reshape(-1)))
    print(f"\nExample {example_idx + 1} Overall Metrics:")
    print(f"  MAE:  {example_mae:.6f}")
    print(f"  RMSE: {example_rmse:.6f}")
    print("-" * 50)